In [ ]:
import spacy
from spacy.language import Language
import fr_core_news_sm
from spacy_langdetect import LanguageDetector
from sklearn.linear_model import LogisticRegression
import pandas as pd
nlp = spacy.load('fr_core_news_sm')
nlp.add_pipe('language_detector_fr2', last=True)

@Language.factory("language_detector_fr2")
def get_lang_detector(nlp, name):
    return LanguageDetector()

In [ ]:
sentence = "Je veux aller à Paris de Marseille"
doc = nlp(sentence)
language_utilise = doc._.language

Capitals = dict()
num = 0
for ent in doc.ents:
    if(num == 0):
        Capitals['depart'] = ent.text
    elif(num == len(Capitals)):
        Capitals['destination'] = ent.text
    num += 1
data.loc[0, 'Ville_1'] = 0
data.loc[0, 'Ville_2'] = 1
data.loc[0, 'inputs'] = sentence
data.loc[1, 'inputs'] = "Je veux aller de Paris à Marseille"
data.loc[1, 'Ville_1'] = 1
data.loc[1, 'Ville_2'] = 0
print(data)
language_utilise

In [ ]:
from spacy.lang.fr import French
import spacy
nlp = spacy.load('fr_core_news_md')
import string
import re

def lemmatizer(text):
    
    text_list = []
    tokens = nlp(text)
    lemma_list = []
    
    for token in tokens:
        if token in nlp.Defaults.stop_words:
            continue
        lemma_list.append(token.lemma_)

    return lemma_list

def preprocessor(text):
    if isinstance((text), (str)):
        text = text.replace('\r\n', ' ')
        text = re.sub("@\S+", "", text.lower())
        text = re.sub(r'http\S+', '', text)
        text = text.translate(str.maketrans('', '', string.punctuation))
        text = re.sub("[’«–…]", ' ', text)
        text = text[:-15]
    
        return text
    if isinstance((text), (list)):
        return_list = []
        for i in range(len(text)):
            temp_text = text.replace('\r\n', ' ')
            temp_text = re.sub("@\S+", "", temp_text.lower())
            temp_text = re.sub(r'http\S+', '', temp_text)
            temp_text = temp_text.translate(str.maketrans('', '', string.punctuation))
            temp_text = temp_text[:-15]
            return_list.append(temp_text)
        return(return_list)
    raise ValueError()
    
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

tfidf_vectorizer = TfidfVectorizer(max_df=1, min_df=0,
                                   use_idf=True, preprocessor=preprocessor, 
                                   tokenizer=lemmatizer, ngram_range=(2, 2))

X = tfidf_vectorizer.fit_transform(data['inputs'])

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
history = model.fit(x=X.todense(), y=data.iloc[:, 3:].values, epochs=1000, verbose=False)


In [ ]:
test = tfidf_vectorizer.transform(["direction Paris en quittant de Marseille"])
model.predict(test.todense())